### Import thư viện cần thiết

In [13]:
import requests
import datetime
from selenium.webdriver.common.action_chains import ActionChains
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

### Tạo Webdriver

In [5]:
apart_arr = []
# Declare browser
options = Options()
options.add_argument("--headless")
options.add_argument("---window-size=1920,1080")
PATH = "D:\Tai_Phan_Mem\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(PATH, options = options)

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\4132057042.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(PATH, options = options)


### Khởi tạo Dataframe chứa dữ liệu thu thập

In [36]:
# DATAFRAME FEATURES
# address, price, area, property_status, price_per_m2, bedroom, toilets, floornumber, property_legal, furniture
df_columns = ['address', 'area', 'price', 'property_status', 'price_per_m2', 'bedroom', 'toilets', 'floornumber','property_legal', 'furniture']
len(df_columns)
# Khởi tạo dataframe rỗng, tạo sẵn các cột
df = pd.DataFrame(columns = df_columns)
df.drop(df.index,inplace=True) 
df.shape

(0, 10)

### 1. Lấy link (đường dẫn) đến các chung cư 

In [28]:
# hàm lấy link của các trang chủ (num_page = số trang)
# Thêm vào
def get_apart_links(url, num_page):
    apartments_link = []
    for i in range(num_page):
        driver2 = webdriver.Chrome(PATH) #, options = options)
        driver2.get(url + str(i))
        
        # Open URL
        soup = BeautifulSoup(driver2.page_source)
        a = soup.find_all("a",class_='AdItem_adItem__gDDQT')
        for i in range(len(a)):
            link ="https://nha.chotot.com/"+a[i]['href']
            apartments_link.append(link)
    return apartments_link


In [27]:
url = "https://www.nhatot.com/mua-ban-can-ho-chung-cu-tp-ho-chi-minh?page="
apartment_links = get_apart_links(url, 1)
print('Số lượng chung cư sẽ lấy thông tin:', len(apartment_links))

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\260860081.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(PATH, options = options)


Số lượng chung cư sẽ lấy thông tin: 0


In [29]:
url = "https://www.nhatot.com/mua-ban-can-ho-chung-cu-tp-ho-chi-minh?page="
num_page = 58
apartment_links = get_apart_links(url, num_page)
print('Số lượng chung cư sẽ lấy thông tin:', len(apartment_links))

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\3122519043.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver2 = webdriver.Chrome(PATH) #, options = options)


Số lượng chung cư sẽ lấy thông tin: 1175


In [32]:
apart_link_df = pd.DataFrame(data = apartment_links)
apart_link_df.to_csv('apart_link-1.csv', index = False)
apart_link_df.shape

(1175, 1)

### 2. Truy cập từng chung cư để lấy dữ liệu

In [33]:
# Trả về thông tin (list) của 1 apartment
def get_info_of_apartment(apartment_link):
    driver1 = webdriver.Chrome(PATH) #, options = options)
    driver1.get(apartment_link)
    driver1.execute_script('window.scrollTo(0,700)')

    try:
        xemthem = driver1.find_element(By.XPATH, '//*[@id="__next"]/div/div[3]/div[1]/div/div[4]/div/div[3]/button')
        xemthem.click()
    except: 
        pass
    soup1 = BeautifulSoup(driver1.page_source)
    
    price = soup1.find('span', itemprop = 'price')
    price = price.text.split(' - ')[0]
    area = soup1.find('span', itemprop = 'size')
    address = soup1.find(class_ = 'fz13')
    property_status = soup1.find('span', itemprop = 'property_status')
    price_per_m2 = soup1.find('span', itemprop="price_m2")
    bedroom = soup1.find('span', itemprop="rooms")
    toilets = soup1.find('span', itemprop="toilets")
    floornumber = soup1.find('span', itemprop="floornumber")
    property_legal = soup1.find('span', itemprop="property_legal_document")
    furniture = soup1.find('span', itemprop = 'furnishing_sell')
    
    info_list = [] # HTML type
    info_text = [] # text
    info_list.extend([address, price, area, property_status, price_per_m2, bedroom, toilets, floornumber, property_legal, furniture])
    info_text.extend([address.text[:-10], price])

    for feature in info_list[2:]:
        if feature != None:
            info_text.append(feature.text)
        else:
            info_text.append('')
    driver1.quit()
    return info_text
    


In [39]:
# Lấy thông tin từng trang, thêm vào df

def get_info_of_page(apart_arr):
    df_temp = df.copy()
    for apart in apart_arr:
        try:
            apart_info = get_info_of_apartment(apart)
            df_temp.loc[len(df_temp)] = apart_info
        except:
            continue
    return df_temp
print(len(apartment_links))
# get_info_of_page(apartment_links)        

1175


In [41]:
df1 = get_info_of_page(apartment_links[0:100])
df1.to_csv('chotot-data-1.csv', index = False)
df1.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

(96, 10)

In [42]:
df2 = get_info_of_page(apartment_links[100:200])
df2.to_csv('chotot-data-2.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

In [43]:
df3 = get_info_of_page(apartment_links[200:300])
df3.to_csv('chotot-data-2.csv', index = False)
df3.shape

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

(100, 10)

In [44]:
df4 = get_info_of_page(apartment_links[300:400])
df4.to_csv('chotot-data-4.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

In [45]:
df5 = get_info_of_page(apartment_links[400:500])
df5.to_csv('chotot-data-5.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

In [46]:
df6 = get_info_of_page(apartment_links[500:600])
df6.to_csv('chotot-data-6.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

In [47]:
df7 = get_info_of_page(apartment_links[600:700])
df7.to_csv('chotot-data-7.csv', index = False)


C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

In [48]:
df8 = get_info_of_page(apartment_links[700:800])
df8.to_csv('chotot-data-8.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

In [50]:
df9 = get_info_of_page(apartment_links[800:900])
df9.to_csv('chotot-data-9.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

In [51]:
df10 = get_info_of_page(apartment_links[900:1000])
df10.to_csv('chotot-data-10.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

In [52]:
df11 = get_info_of_page(apartment_links[1000:1100])
df11.to_csv('chotot-data-11.csv', index = False)

C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver1 = webdriver.Chrome(PATH) #, options = options)
C:\Users\Admin\AppData\Local\Temp\ipykernel_34056\934555059.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service obje

In [57]:
df_list = [df1, df2, df3, df4, df5, df6, df7, df8, df9, df10, df11]
df = pd.concat(df_list, axis = 0)
df.shape

(1094, 10)

In [58]:
# Xuất file csv
df.to_csv('chotot-data.csv', index = False)

In [59]:
df.head(10)

,address,area,price,property_status,price_per_m2,bedroom,toilets,floornumber,property_legal,furniture
0,"Số 4, Đào Trí, Phường Phú Thuận, Quận 7, Tp Hồ...","2,3 tỷ",69 m²,Đã bàn giao,"33,33 triệu/m²",2 phòng,2 phòng,,,
1,"996, 996 Đường Tạ Quang Bửu, Phường 6, Quận 8,...","1,2 tỷ",52 m²,Chưa bàn giao,"23,08 triệu/m²",2 phòng,,,,
2,"Bàu Cát 7, Phường 14, Quận Tân Bình, Tp Hồ Chí...","1,55 tỷ",42 m²,Đã bàn giao,"36,9 triệu/m²",1 phòng,,,Đã có sổ,
3,"I142, Đường Số 9A, Xã Bình Hưng, Huyện Bình Ch...","4,8 tỷ",150 m²,Đã bàn giao,32 triệu/m²,3 phòng,2 phòng,14,Đã có sổ,Nội thất đầy đủ
4,"Hậu Giang, Phường 11, Quận 6, Tp Hồ Chí Minh","2,1 tỷ",82 m²,Chưa bàn giao,"25,61 triệu/m²",2 phòng,2 phòng,,Đã có sổ,Hoàn thiện cơ bản
5,"Đường Điện Biên Phủ, Phường 22, Bình Thạnh, Hồ...",11 tỷ,109 m²,Chưa bàn giao,"100,92 triệu/m²",3 phòng,2 phòng,,Đã có sổ,Nội thất cao cấp
6,"961, 961 Đường Hậu Giang, Phường 11, Quận 6, T...","4,5 tỷ",137 m²,Đã bàn giao,"32,85 triệu/m²",3 phòng,3 phòng,18,Đã có sổ,Hoàn thiện cơ bản
7,"Hiệp Bình Chánh, Phường Hiệp Bình Chánh (Quận ...","3,42 tỷ",71 m²,Đã bàn giao,"48,17 triệu/m²",2 phòng,,17,,
8,"Hiệp Bình Chánh, Phường Hiệp Bình Chánh (Quận ...","3,4 tỷ",71 m²,Đã bàn giao,"47,89 triệu/m²",2 phòng,2 phòng,17,,
9,"Đường Nguyễn Hữu Thọ, Xã Phước Kiển, Huyện Nhà...","3,19 tỷ",121 m²,Chưa bàn giao,"26,36 triệu/m²",3 phòng,2 phòng,23,Đã có sổ,Hoàn thiện cơ bản
